In [1]:
import torch as t
from transformers import LlamaForCausalLM, LlamaTokenizer
import argparse
import pandas as pd
from tqdm import tqdm
import os
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
LLAMA_DIRECTORY = config['LLaMA']['weights_directory']

if not os.path.exists(LLAMA_DIRECTORY):
    raise Exception("Make sure you've set the path to your LLaMA weights in config.ini")

class Hook:
    def __init__(self):
        self.out = None

    def __call__(self, module, module_inputs, module_outputs):
        self.out, _ = module_outputs


def load_llama(model_size, device):
    llama_path = os.path.join(LLAMA_DIRECTORY, config['LLaMA'][f'{model_size}_subdir'])
    print(llama_path)
    tokenizer = LlamaTokenizer.from_pretrained(llama_path, device_map='cuda')
    model = LlamaForCausalLM.from_pretrained(llama_path, device_map='cuda')
    # set tokenizer to use bos token
    tokenizer.bos_token = '<s>'
    # if model_size == '13B' and device != 'cpu':
    #     model = model.half()
    model.to(device)
    return tokenizer, model

def load_statements(dataset_name):
    """
    Load statements from csv file, return list of strings.
    """
    dataset = pd.read_csv(f"datasets/{dataset_name}.csv")
    statements = dataset['statement'].tolist()
    return statements

In [2]:
tokenizer, model = load_llama('13B-2', 'cuda:0')
all_statements = load_statements('cities')
device='cuda:0'
layers=[0]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


/storage/models/llama-2-13B-GPTQ-4bit


In [3]:
statements=all_statements[:1]

# attach hooks
hooks, handles = [], []
for layer in layers:
    hook = Hook()
    handle = model.model.layers[layer].register_forward_hook(hook)
    hooks.append(hook), handles.append(handle)

# get activations
acts = {layer : [] for layer in layers}
for statement in tqdm(statements):
    input_ids = tokenizer.encode(statement, return_tensors="pt").to(device)
    model(input_ids)
    for layer, hook in zip(layers, hooks):
        acts[layer].append(hook.out[0, -1])

for layer, act in acts.items():
    acts[layer] = t.stack(act).float()

# remove hooks
for handle in handles:
    handle.remove()

100%|██████████| 1/1 [00:00<00:00,  2.00it/s]


In [5]:
acts[0][0]

tensor([ 0.0133, -0.0036,  0.0015,  ...,  0.0061,  0.0028,  0.0415],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [6]:
input_ids

tensor([[    1,   450,  4272,   310,   476,  3417, 29876,   397,   279,   338,
           297, 12710, 29889]], device='cuda:0')

In [7]:
outputs=model(input_ids, output_hidden_states=True)

In [16]:
outputs.hidden_states[1][0][-1][:]

tensor([ 0.0133, -0.0036,  0.0015,  ...,  0.0061,  0.0028,  0.0415],
       device='cuda:0', dtype=torch.float16, grad_fn=<SliceBackward0>)

In [93]:
outputs.hidden_states[15]

tensor([[[ 0.2035, -0.4111,  0.2014,  ..., -0.0191,  0.3267, -0.5210],
         [-0.1396, -0.7192,  0.0668,  ..., -0.0035,  0.3130,  0.3230],
         [-0.1270,  0.1328, -0.0320,  ...,  0.3794,  0.2050, -0.6992],
         ...,
         [-0.1923, -0.1631, -0.2490,  ...,  0.2705,  0.1188, -0.2795],
         [-0.1245, -0.1892, -0.2700,  ..., -0.2158,  0.1249, -0.9170],
         [ 0.0220, -0.2148, -0.4954,  ..., -0.1597, -0.0896, -0.1294]]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddBackward0>)